<a href="https://colab.research.google.com/github/vanderbilt-ml/51-callahan-mlproj-realestate/blob/feature-EDA/real_estate_project_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real Estate Prediction and Market Intelligence

### Background
Real estate has many opportunities in which to apply data science methods.  I want to develop a model that will allow for identification of pricing assymetries in the real estate market by being able to accurately price assets and there by identify opportunities when new properties are discovered. 

### Project Description
My goal is to develop a model that can detect "good prices" in real estate markets of my choosing in order to identify deals.  Maybe I will buy one of these properties someday!

### Performance Metric + Analyses
1. *Pricing Accuracy* - can the model learn what a good price is and make a price prediction for a specific property?  This would involve predicting the price at which a property will sell based on a variety of features, and can be verified in training data by comparing the prediction with the actual sale price.  RMSE comes to mind as a possible measure of model accuracy. 

2. *Value Drivers* - which attributions associated with a property influence price and do those factors change based on geography?  Obvious things like square-foot (size) of property will matter, but there are likely other discoverable features here that provide a rich opportunity for feature engineering.  Normalizing for macro-economic trends may also be an option, since housing prices are defined inside a marketplace that is influenced by factors like: mortgage rates, inflation, GDP, etc... 

3. *Similar markets* - Which markets are complementary in the sense that they have similar attributes and properties represent a similar value to purchasers?  Identifying a similarity metric for markets would allow this analysis to encompass greater quantities of data.  Presumably, data from similar markets could be combined to make predictions while dissimilar ones should be excluded.  As an example, California beachfront property or NYC apartments are likely not comparable to a single family home in the midwest and we would not want to train data using Cali/NYC data if we want to predict home prices in rural Kansas.

In [25]:
import pandas as pd
import numpy as np


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
eda = pd.read_csv('https://raw.githubusercontent.com/vanderbilt-ml/51-callahan-mlproj-realestate/main/northern_MI_market_total_2022-05-22.csv?token=GHSAT0AAAAAABULIU4HXPL5F5Y4B7BO2XTKYUSGQAA')

In [4]:
eda.head(2)

,property_id,prop_type,last_update,address,address_new,prop_status,price_raw,sqft_raw,is_showcase,is_new_construction,...,agent_name,agent_photo,office_name,office_advertiser_id,price_reduced,products,price_reduced_date,is_cobroker,is_foreclosure,snapshot_date
0,3586054640,mobile,3/11/22,"4439 Ironwood Dr N, Traverse City, 49685","{'line': '4439 Ironwood Dr N', 'street_number'...",recently_sold,10000,732.0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5/22/22
1,4501321015,single_family,3/11/22,"4783 Cedar Knoll Dr, Traverse City, 49685","{'line': '4783 Cedar Knoll Dr', 'street_number...",recently_sold,11000,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5/22/22


Questions: 

**Are the enough data points to perform a prediction in the geographical area of interest?**

*Yes, we have enough data to begin, and can collect more as needed.*

**What data cleaning tasks are necessary?**

*Data conversions, replacing NANs and strings, and strategically dropping some categories/columns.  Dropping foward looking values is important to avoiding leakage.*

**Is missing data a problem for any particular columns?**

*Yes, key missing values for sqft and bathrooms pose a risk.  Will need to be cautious with filling those.*

**Do any basic patterns emerge that increase/decrease trust in the dataset?**

*None that cannot be handled with data cleaning.*


In [90]:
# Question 1  
# 944 entries, 141 land (vacant lots) as well as other/commercial and farm can all be dropped.  These are non-residential properties and not the focus of this analysis. 
# 789 values in dataset.  Conclusion - small but workable, can expand information collection to neighboring areas if necessary.

display(eda.shape[0])
display(eda['prop_type'].value_counts())

properties_only = eda[ eda['prop_type'].isin(['single_family', 'condo', 'multi_family', 'mobile']) ]

display(properties_only.shape[0])
display(properties_only['prop_type'].value_counts())

944

single_family    475
condo            286
land             141
mobile            20
other             10
multi_family       8
commercial         3
farm               1
Name: prop_type, dtype: int64

789

single_family    475
condo            286
mobile            20
multi_family       8
Name: prop_type, dtype: int64

In [91]:
# Question 2

properties_only.isnull().sum()

property_id               0
prop_type                 0
last_update               0
address                   0
address_new               0
prop_status               0
price_raw                 0
sqft_raw                120
is_showcase               0
is_new_construction       0
price                     0
beds                    190
baths                     0
sqft                      0
lot_size                160
photo                     0
short_price               0
baths_half              610
baths_full              155
photo_count             159
sold_date                 0
lat                      60
lon                      60
has_leadform              0
page_no                   0
rank                      0
list_tracking             0
listing_id              276
list_date               276
advertiser_id           308
agent_name              287
agent_photo             417
office_name             276
office_advertiser_id    339
price_reduced           276
products            

In [92]:
leakage_cols = ['price_reduced', 'price_reduced_date', 'property_id']

website_cols = ['rank', 'list_tracking', 'page_no', 'photo']

redundant_cols = ['sqft', 'price', 'address', 'short_price']

properties_only = properties_only.drop(columns=leakage_cols + website_cols + redundant_cols)

In [93]:
# Bathrooms - likely price driver, needs cleaning

In [94]:
properties_only[['baths_half', 'baths_full', 'baths']].query('baths_half == baths_half').head(5)

,baths_half,baths_full,baths
52,1.0,1.0,2
124,1.0,1.0,2
132,1.0,1.0,2
150,1.0,1.0,2
151,1.0,1.0,2


In [95]:
properties_only['bath'] = properties_only['baths_half'].fillna(0) / 2 +  properties_only['baths_full'].fillna(0)

In [96]:
# Remove embedded strings, NAN better
properties_only['bath'] = np.where(properties_only['baths'] == 'Call', np.nan, properties_only['baths'])

In [97]:
properties_only = properties_only.drop(columns=['baths_half', 'baths_full', 'baths'])

In [98]:
# Extract lot sizes, convert to same metrics

def acres_to_sqft(row):
  if 'acres' in str(row['lot_size']):
    return float(str.strip(row['lot_size'].replace('acres', '').replace(',', ''))) * 43560
  elif 'sq ft lot' in str(row['lot_size']):
    return float(str.strip(row['lot_size'].replace('sq ft lot', '').replace(',', '')))
  elif pd.isna(row['lot_size']):
    return row['lot_size']   
  else:
    return -1   

properties_only['lot_size_sqft'] = properties_only.apply(acres_to_sqft, axis=1)

In [99]:
properties_only[ properties_only['lot_size_sqft'] == -1]

,prop_type,last_update,address_new,prop_status,price_raw,sqft_raw,is_showcase,is_new_construction,beds,lot_size,photo_count,sold_date,lat,lon,has_leadform,listing_id,list_date,advertiser_id,agent_name,agent_photo,office_name,office_advertiser_id,products,is_cobroker,is_foreclosure,snapshot_date,bath,lot_size_sqft


In [ ]:
# TODO: get binary values into correct format (1,0) vs TRUE FALSE or TRUE NAN
# Extract address new to get zip codes

In [60]:
properties_only['beds'].value_counts(dropna=False, normalize=True)

3.0     0.307985
NaN     0.240811
2.0     0.186312
4.0     0.169835
1.0     0.043093
5.0     0.040558
6.0     0.008872
8.0     0.001267
10.0    0.001267
Name: beds, dtype: float64